In [7]:
import cv2
import numpy as np
import os

face_model = {
    "proto": "models/opencv_face_detector.pbtxt",
    "model": "models/opencv_face_detector_uint8.pb"
}
age_model = {
    "proto": "models/age_deploy.prototxt",
    "model": "models/age_net.caffemodel"
}
gender_model = {
    "proto": "models/gender_deploy.prototxt",
    "model": "models/gender_net.caffemodel"
}

face_net = cv2.dnn.readNet(face_model["model"], face_model["proto"])
age_net = cv2.dnn.readNet(age_model["model"], age_model["proto"])
gender_net = cv2.dnn.readNet(gender_model["model"], gender_model["proto"])

AGE_LIST = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
GENDER_LIST = ['Male', 'Female']

image_folder = "images/" 
padding = 20

for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)
    frame = cv2.imread(img_path)
    if frame is None:
        print(f"Could not read {img_name}")
        continue

    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104, 117, 123), False)
    face_net.setInput(blob)
    detections = face_net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.6:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x1, y1, x2, y2) = box.astype(int)

            face = frame[max(0, y1-padding):min(h, y2+padding), max(0, x1-padding):min(w, x2+padding)]
            if face.size == 0:
                continue

            face_blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), [104, 117, 123], swapRB=False)

            gender_net.setInput(face_blob)
            gender = GENDER_LIST[gender_net.forward().argmax()]

            age_net.setInput(face_blob)
            age = AGE_LIST[age_net.forward().argmax()]

            label = f"{gender}, {age}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    cv2.imshow("Result", frame)
    cv2.waitKey(0)

cv2.destroyAllWindows()
